<a href="https://colab.research.google.com/github/Adminixtrator/gpt-2/blob/master/GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calling file from Repository




In [1]:
!git clone https://github.com/adminixtrator/gpt-2.git

%cd gpt-2
%ls

Cloning into 'gpt-2'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.
/content/gpt-2
Dockerfile.cpu  domains.txt        LICENSE        README.md         src/
Dockerfile.gpu  download_model.py  model_card.md  requirements.txt


# Using the gpt-2 model 345M

In [2]:
#Download the gpt-2 model 345M..
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 577kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 34.2Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 646kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:29, 48.1Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 6.46Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 30.3Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 22.6Mit/s]                                                       


In [0]:
#Encoding..
!export PYTHONIOENCODING=UTF-8

# Now to Implementing gpt-2

In [0]:
#Changing directory..
import os

os.chdir('src')

In [0]:
#Importing the necessary libraries..

import json
import numpy as np
import tensorflow as tf
import model, sample, encoder

In [0]:
#Function to use the interaction model..

def interact_model(model_name, seed, nsamples, batch_size, length, temperature, top_k, models_dir):
  
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(hparams=hparams, length=length, context=context, batch_size=batch_size, temperature=temperature, top_k=top_k)

        saver = tf.train.Saver(save_relative_paths=True)
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("\nModel prompt >>> ")
            if raw_text == 'ADMIN_NIXTRATOR':
              raw_text = False
              break
            while not raw_text:
                print('\nPrompt should not be empty!')
                raw_text = input("\nModel prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)
            
            

# **Code Explanation**

## **model_name**: 
 This indicates which model we are using. In our case, we are using the GPT-2 model with 345 million parameters or weights

## **seed**: 
Integer seed for random number generators, fix seed to reproduce results

## **nsamples**: 
This represents the number of sample texts generated in our output

## **batch_size**: 
This only affects speed/memory. This must also divide nsamples

*Note: To generate more than one sample, you need to change the values of both nsamples and batch_size and also have to keep them equal.*

## **length**: 
It represents the number of tokens in the generated text. If the length is None, then the number of tokens is decided by model hyperparameters

## **temperature**: 
This controls randomness in Boltzmann distribution. Lower temperature results in less random completions. As the temperature approaches zero, the model will become deterministic and repetitive. Higher temperature results in more random completions

## **top_k**:  
This parameter controls diversity. If the value of top_k is set to 1, this means that only 1 word is considered for each step (token). If top_k is set to 40, that means 40 words are considered at each step. 0 (default) is a special setting meaning no restrictions. top_k = 40 generally is a good value

## **models_dir**: 
It represents the path to parent folder containing model subfolders (contains the <model_name> folder)

# Results

In [13]:
#Using the arguements above..

interact_model('345M', None, 1, 1, 100, 1, 0, '/content/gpt-2/models')

INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt

Model prompt >>> Yesterday was my birthday
======================================== SAMPLE 1 ========================================
.

So, obviously, I've been living with this abandonment ever since I relapsed about five years ago, and my boyfriend (who was recovering from being on anti-depressants) once tried to fill in the blanks after my abandoned sister found out about the realtered pattern: "If only I had somehow gotten pregnant when I was 7, 6, 5, 4…" with cute fake prosthetic babies, all for the sake of making my life better! I knew getting a

Model prompt >>> He asked me out and
======================================== SAMPLE 1 ========================================
 I said no!", she tells Maryanne. "For the next twelve years I was in a relationship with someone else. I was really content to remain single, keeping everything to myself. I think my anxiety has managed to overwhelm me with though